# Project 8
### Try Detectron 2

Import Library

In [1]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode

In [ ]:
def get_cat_dict(directory,jsonfile_name):
    classes = ['Khawtang','Khawniewping']
    dataset_dicts = []
    json_file = os.path.join(directory,jsonfile_name)
    with open(json_file) as f:
        json_ann_file = json.load(f)
    
    image_list = json_ann_file['images']

    for image in image_list :
        filename = os.path.join(directory,img_anns[''])